Travel and Expenses - Advanced techniques

In [1]:
#!Python3

import pandas as pd
import os 

from time import gmtime, strftime
import datetime

#XLS reader library
import xlrd  


In [6]:
#This piece is for azure notebooks to load the test data from github
!ls
!mkdir testdata
!curl https://raw.githubusercontent.com/account/giitrepo/testdata/example_expenses01.xlsx -o testdata/test.xlsx
!ls testdata

Generate Test Data.ipynb	  hello world.ipynb
Travel and Expenses review.ipynb  testdata
mkdir: cannot create directory 'testdata': File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20838  100 20838    0     0  61299      0 --:--:-- --:--:-- --:--:-- 62389
test.xlsx


We open the Excel file and list the sheets inside, just to check we are reading it properly

In [9]:
book = xlrd.open_workbook('testdata/test.xlsx')
n= book.nsheets
for x in range(0 , n):
    print (x)
    print ("Sheets " + book.sheet_names()[x])



0
Sheets AuditTrips
1
Sheets ExpenseType


In [10]:
#Here we load all the data into dataframes 
df = pd.DataFrame([])
ts= strftime("%Y-%m-%d %H:%M:%S")
input_frames=[]
for x in range(0, n):
    sh=book.sheet_by_index(x)
    cellA1= sh.cell_value(0,0)
    print ("Processing sheet " + book.sheet_names()[x])
    if cellA1=="Name":  
        num_cols = sh.ncols   # Number of columns
        num_rows = sh.nrows
        print (num_cols, num_rows)
        data=[]
        for row_idx in range(0, num_rows):    # Iterate through rows use sh.nrows for all rows    
            row_data=[]
            for col_idx in range(0, num_cols):  # Iterate through columns
                
                
                
                cell_obj = sh.cell(row_idx, col_idx)  # Get cell object by row, col
                cell_value= cell_obj.value
                cell_type = sh.cell_type(row_idx, col_idx)
                
                if cell_type == xlrd.XL_CELL_DATE:
                    get_col= xlrd.xldate.xldate_as_datetime (cell_value, book.datemode)
                elif cell_type == xlrd.XL_CELL_NUMBER:
                    get_col = float(cell_value)
                else:
                    get_col = str(cell_value)
               
                row_data.append(get_col)
            data.append(row_data)
            
        #We create a frame with the first row beign the column names
        frame=pd.DataFrame(data[1:],columns=data[0])
        #We add here a column for the source sheet and a timestamp
        frame["SourceSheet"]=sh.name
        frame["Timestamp"]= ts
        input_frames.append (frame)
        print(list(frame))
    else:
        print("Not an expenses extracct, skipping")

Processing sheet AuditTrips
7 145
['Name', 'Title', 'Expenses', 'Date', 'Date Claim', 'CostCentre', 'ExpenseType', 'SourceSheet', 'Timestamp']
Processing sheet ExpenseType
Not an expenses extracct, skipping


In [12]:
sh=book.sheet_by_name('ExpenseType')
num_cols = sh.ncols   # Number of columns
num_rows = sh.nrows
print (num_cols, num_rows)
data=[]
for row_idx in range(0, num_rows):    # Iterate through rows use sh.nrows for all rows    
    row_data=[]
    for col_idx in range(0, num_cols):  # Iterate through columns
        cell_obj = sh.cell(row_idx, col_idx)  # Get cell object by row, col
        cell_value= cell_obj.value
        cell_type = sh.cell_type(row_idx, col_idx)        
        if cell_type == xlrd.XL_CELL_DATE:
            get_col= xlrd.xldate.xldate_as_datetime (cell_value, book.datemode)
        elif cell_type == xlrd.XL_CELL_NUMBER:
            get_col = float(cell_value)
        else:
            get_col = str(cell_value)

        row_data.append(get_col)
        data.append(row_data)
df_expensetype=pd.DataFrame(data[1:],columns=data[0])
print(list(df_expensetype))

2 14
['ExpenseType', 'Description']


In [13]:
#Here we concatenate the multiple dataframes we created , one per sheet
# the ignore_index option is so that the main index is recreated so each row will have a unique ID 
if len(input_frames) >0:   
    df = pd.concat(input_frames, ignore_index=True)

df['Expenses'] = pd.to_numeric(df['Expenses'])
df['Expenses']=df['Expenses'].fillna(0) #To convert any blanks/NaN to zero, you may want to check those before killing them

print ("Top 10 records")
print (df[0:10])



Top 10 records
     Name    Title  Expenses       Date Date Claim  CostCentre  ExpenseType  \
0    John   Senior    1000.0 2014-08-15 2014-09-24      1001.0         12.0   
1   Peter  Manager    1500.0 2014-07-01 2014-08-30      1001.0          4.0   
2    Arne   Senior      -1.0 2014-07-01 2014-08-10      1001.0          9.0   
3   Hiten   Senior       0.0 2015-01-01 2015-02-10      1001.0          8.0   
4  Najwan   Senior     830.0 2015-10-01 2015-11-10      1001.0          5.0   
5   Peter  Manager    3000.0 2015-11-30 2016-01-09      1001.0          5.0   
6    John   Senior      14.0 2014-08-15 2014-09-24      1001.0         13.0   
7   Peter  Manager    4436.0 2014-07-01 2014-08-30      1001.0          7.0   
8    Arne   Senior     299.0 2014-07-01 2014-08-10      1001.0         11.0   
9   Hiten   Senior      34.0 2015-01-01 2015-02-10      1001.0         12.0   

  SourceSheet            Timestamp  
0  AuditTrips  2018-02-03 14:36:35  
1  AuditTrips  2018-02-03 14:36:35  
2  A

In [14]:
print ("Record count (non null):")
print(df.count())

Record count (non null):
Name           144
Title          144
Expenses       144
Date           144
Date Claim     144
CostCentre     144
ExpenseType    144
SourceSheet    144
Timestamp      144
dtype: int64
Min value:-1.0
Max value:4436.0


In [21]:
# This is an optional command to set the index, if we had some column that can act as index, otherwise, we get the default row number  on import
#df.set_index("Name", inplace=True, drop=False)


In [27]:
# Create a Pandas Excel writer using XlsxWriter as the engine.

# Delete any previous output
try:
    os.remove('testdata/test_duplicates.xlsx')
except:
    #Do nothing we dont have the file
    print ("Nothing to delete")
    
    
!ls testdata

Nothing to delete
test.xlsx


In [28]:
#We write the new excel , dumping the dataframe
try:
    writer = pd.ExcelWriter('testdata/test_duplicates.xlsx', engine='xlsxwriter')
    dupli_df.to_excel(writer, sheet_name='Sheet1')
except:
    print ("error creating/saving excel file")
!ls testdata

test.xlsx  test_duplicates.xlsx


In [29]:
# Now we just open it to test it worked

book = xlrd.open_workbook('testdata/test_duplicates.xlsx')


In [30]:
xl_sheet = book.sheet_by_index(0)
#Alternatively:
#xl_sheet = book.sheet_by_name('Sheet1')

num_cols = xl_sheet.ncols   # Number of columns

for row_idx in range(0, xl_sheet.nrows):    # Iterate through rows
    print ('-'*40)
    print ('Row: %s' % row_idx)   # Print row number
    for col_idx in range(0, num_cols):  # Iterate through columns
        cell_obj = xl_sheet.cell(row_idx, col_idx)  # Get cell object by row, col
        print ('Column: [%s] cell_obj: [%s]' % (col_idx, cell_obj))


----------------------------------------
Row: 0
Column: [0] cell_obj: [empty:'']
Column: [1] cell_obj: [text:'Name']
Column: [2] cell_obj: [text:'Title']
Column: [3] cell_obj: [text:'Expenses']
Column: [4] cell_obj: [text:'Date']
Column: [5] cell_obj: [text:'Date Claim']
Column: [6] cell_obj: [text:'CostCentre']
Column: [7] cell_obj: [text:'ExpenseType']
Column: [8] cell_obj: [text:'SourceSheet']
Column: [9] cell_obj: [text:'Timestamp']
----------------------------------------
Row: 1
Column: [0] cell_obj: [number:152.0]
Column: [1] cell_obj: [text:'Michaela']
Column: [2] cell_obj: [text:'Senior']
Column: [3] cell_obj: [number:300.0]
Column: [4] cell_obj: [xldate:42114.0]
Column: [5] cell_obj: [xldate:42124.0]
Column: [6] cell_obj: [number:1002.0]
Column: [7] cell_obj: [number:9.0]
Column: [8] cell_obj: [text:'Events']
Column: [9] cell_obj: [text:'2017-02-06 20:45:25']
----------------------------------------
Row: 2
Column: [0] cell_obj: [number:271.0]
Column: [1] cell_obj: [text:'Micha